### Supermarket data science case study - Exploring first data


### Importing packages

In [9]:
import pandas as pd
import sys
import matplotlib.pyplot as plt
import altair as alt
import vegafusion as vf
import sklearn
import vega_datasets
from sklearn.pipeline import Pipeline, make_pipeline
import seaborn as sns
from matplotlib.patches import Patch
import numpy as np


### Functions

In [10]:
def f_concat(l_input):

    # Initialize.
    dummy = ""
    n_len = len(l_input)

    if n_len == 1:
        return l_input[0]

    # Loop through text elements.
    for i in range(n_len - 1):
        dummy = dummy + l_input[i] + ", "

    # Append last element.
    dummy = dummy + "and " + l_input[n_len - 1]

    # Return result.
    return dummy

In [11]:
def f_describe(df_input, n_top=10):

    print("First " + str(n_top) + " rows in de data:")
    display(df_input.head(n_top))

    df_numeric = df_input.select_dtypes(
        include=[
            "uint8",
            "uint16",
            "uint32",
            "uint64",
            "int8",
            "int16",
            "int32",
            "int64",
            "float16",
            "float32",
            "float64",
        ]
    )

    if len(df_numeric.columns):
        print("Numerical data:")
        display(df_numeric.describe())

    df_textual = df_input.select_dtypes(include=["category", "object", "bool"])

    if len(df_textual.columns):
        print("Textual data:")
        display(df_textual.describe())

    v_na = [
        col
        + " ("
        + str(df[col].isna().sum())
        + ", "
        + str(round(100 * df[col].isna().sum() / df.shape[0], 1))
        + "%)"
        for col in df.columns
        if df[col].isna().sum() > 0
    ]

    if len(v_na) > 0:
        print("Features and their number of missing values:")
        display(f_concat(v_na))

### Downcast and transform data
Update formatting of features to optimize memory and standardize column names.

In [12]:
def standardize_column_names(s):
    return s.replace(" ", "")


def optimize_memory(df):
    # Change: Objects to Categorical.                                               #WHEN needed to transform Objects to Categorical?
    # object_cols = df.select_dtypes(include="object").columns
    # if not object_cols.empty:
    #     print("Change: Objects to Categorical")
    #     df[object_cols] = df[object_cols].astype("category")

    # Change: Convert integers to smallest unsigned integer and floats to smallest.
    for old, new in [("integer", "unsigned"), ("float", "float")]:
        print("Change: " + old + " --> " + new)
        for col in df.select_dtypes(include=old).columns:
            df[col] = pd.to_numeric(df[col], downcast=new)

    return df


def month_year_to_int(df, i):
    # Change: Month and Year to integer
    if i == 0:
        print("Change: Month and Year to integer")
        df = df.astype({"month": int, "year": int})

    return df

### Transform date-related columns to datetime format.

In [13]:
# Convert datasets to time series
def transform_date_to_datetime(df, i):
    if i == 0:
        print("Change: Transformed 'year', 'month', 'day' columns to Datetime feature")
        df["date"] = pd.to_datetime(df[["year", "month", "day"]], unit="us")

       # print(
       #     "Change: Dropped 'year', 'month', 'day' columns and transformed to Datetime64[us] feature"
       # )
        #df.drop(columns=["day", "month", "year"], inplace=True)

    else:
        if "date" in df.columns:
            print("Change: Transformed 'date' column to Datetime Dtype")
            df["date"] = pd.to_datetime(df["date"]).dt.tz_localize(None)

    return df

### Import data from local PATH
Import data trough pipeline to downcast the data and transformations

In [14]:
def f_get_data(i=0):

    # Define path.
    c_path = "/Users/Georgi/Documents/EAISI/EASI_4B_Supermarket/Group4B/data/raw/"
    # Identify file.
    v_file = (
        "history-per-year",  # 0
        "history_aggregated",  # 1
        "holidays_events",  # 2
        "items",  # 3
        "oil",  # 4
        "stores",  # 5
        "transactions",
    )  # 6

    # Load data.
    df = (
        pd.read_parquet(c_path + v_file[i] + ".parquet")
        .rename(columns=standardize_column_names)
        .pipe(optimize_memory)
        .pipe(month_year_to_int, i)
        .pipe(transform_date_to_datetime, i)
    )

    # Return data.
    return df

### Importing data: Here I import the daily data. The data contains daily sales data excluding holidays

In [15]:
df_salesdata= f_get_data(0)
df_items =f_get_data(3)
df_stores =f_get_data(5)

Change: integer --> unsigned
Change: float --> float
Change: Month and Year to integer
Change: Transformed 'year', 'month', 'day' columns to Datetime feature
Change: integer --> unsigned
Change: float --> float
Change: integer --> unsigned
Change: float --> float


In [16]:
#print(df_stores.info())
#print(df_salesdata.info())
print(df_items.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4100 entries, 0 to 4099
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   item_nbr    4100 non-null   uint32
 1   family      4100 non-null   object
 2   class       4100 non-null   uint16
 3   perishable  4100 non-null   uint8 
dtypes: object(1), uint16(1), uint32(1), uint8(1)
memory usage: 60.2+ KB
None


In [17]:
# Adjust data types and drop columns we don't need
df_salesdata['store_nbr'] = df_salesdata['store_nbr'].astype(str)
df_salesdata = df_salesdata.drop(columns=['year', 'day','onpromotion','month'])
df_stores['store_nbr'] = df_stores['store_nbr'].astype(str)
df_stores['cluster'] = df_stores['cluster'].astype(str)

In [18]:
print(df_stores.info())
print(df_salesdata.info())
print(df_items.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     object
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     object
dtypes: object(5)
memory usage: 2.2+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125497040 entries, 0 to 125497039
Data columns (total 5 columns):
 #   Column      Dtype         
---  ------      -----         
 0   id          uint32        
 1   store_nbr   object        
 2   item_nbr    uint32        
 3   unit_sales  float32       
 4   date        datetime64[ns]
dtypes: datetime64[ns](1), float32(1), object(1), uint32(2)
memory usage: 3.3+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4100 entries, 0 to 4099
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  --

In [21]:
# Group the sales date by store and item
df_salesdatagrouped = df_salesdata.groupby(['store_nbr','date']).agg({'unit_sales':'sum'}).reset_index()

print(f' In df_salesdatagrouped zitten nu {df_salesdatagrouped.shape[0]} rijen en {df_salesdatagrouped.shape[1]} kolommen')
print(df_salesdatagrouped.info())

 In df_salesdatagrouped zitten nu 83606 rijen en 3 kolommen
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83606 entries, 0 to 83605
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   store_nbr   83606 non-null  object        
 1   date        83606 non-null  datetime64[ns]
 2   unit_sales  83606 non-null  float32       
dtypes: datetime64[ns](1), float32(1), object(1)
memory usage: 1.6+ MB
None


In [22]:
df_salesandstoresdata = df_salesdatagrouped.merge(df_stores, left_on='store_nbr', right_on='store_nbr', how='inner')

print(f' In df_salesandstoredata zitten nu {df_salesandstoresdata.shape[0]} rijen en {df_salesandstoresdata.shape[1]} kolommen')
print(df_salesandstoresdata.info())

 In df_salesandstoredata zitten nu 83606 rijen en 7 kolommen
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83606 entries, 0 to 83605
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   store_nbr   83606 non-null  object        
 1   date        83606 non-null  datetime64[ns]
 2   unit_sales  83606 non-null  float32       
 3   city        83606 non-null  object        
 4   state       83606 non-null  object        
 5   type        83606 non-null  object        
 6   cluster     83606 non-null  object        
dtypes: datetime64[ns](1), float32(1), object(5)
memory usage: 4.1+ MB
None


## Step 3.0- Filter out all stores that don't have all the datapoint or atleast mark them


In [23]:
# Count amount of values per store
se_storedatecount = df_salesandstoresdata['store_nbr'].value_counts()


In [24]:
# Create a date range from the start date to the end date of the sales data
start_date = pd.to_datetime('2013-01-02')
end_date = pd.to_datetime('2017-08-15')

# Create a date range variable from the start date to the end date of the sales data
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Create a dataframe from the date range
date_range = pd.DataFrame(date_range, columns=['date'])

print(f'The date_range dataframe starts at {date_range["date"].min()} and ends at {date_range["date"].max()}')

The date_range dataframe starts at 2013-01-02 00:00:00 and ends at 2017-08-15 00:00:00


In [25]:
df_salesandstoresdata34 = df_salesandstoresdata[df_salesandstoresdata['store_nbr'] == '34']

df_salesandstoresdata34missingdates = df_salesandstoresdata34.merge(date_range, left_on='date', right_on='date', how='outer')

empty_unit_sales = df_salesandstoresdata34missingdates[df_salesandstoresdata34missingdates['unit_sales'].isnull()]
print(empty_unit_sales)
print('As we can see, stores that have all data seem to be closed on christmas day and on new years day')

     store_nbr       date  unit_sales city state type cluster
357        NaN 2013-12-25         NaN  NaN   NaN  NaN     NaN
364        NaN 2014-01-01         NaN  NaN   NaN  NaN     NaN
722        NaN 2014-12-25         NaN  NaN   NaN  NaN     NaN
729        NaN 2015-01-01         NaN  NaN   NaN  NaN     NaN
1087       NaN 2015-12-25         NaN  NaN   NaN  NaN     NaN
1094       NaN 2016-01-01         NaN  NaN   NaN  NaN     NaN
1453       NaN 2016-12-25         NaN  NaN   NaN  NaN     NaN
1460       NaN 2017-01-01         NaN  NaN   NaN  NaN     NaN
As we can see, stores that have all data seem to be closed on christmas day and on new years day


In [26]:
# Let's try to make a dataframe that consists of all stores that are missing data for a certain date
# Step 1 - Crossjoin stores with the daterange
df_storesreduced = df_stores.drop(columns=['city', 'state', 'type', 'cluster'])
df_storesanddates = df_storesreduced.merge(date_range, how='cross')

print(f' Now we onstructed a dataframe with all stores and all dates, it contains {df_storesanddates.shape[0]} rows')
print(df_storesanddates.head(5))

# Step 2 - Merge the salesdata with the storesanddates dataframe to have a dataframe consisting of all stores and all dates with unit_sales

df_salesandstoresdata_alldates = df_salesandstoresdata.merge(df_storesanddates, on = ['store_nbr','date'], how='outer')

print(f' Now we onstructed a dataframe with all stores and all dates, it contains {df_salesandstoresdata_alldates.shape[0]} rows')
print(df_salesandstoresdata_alldates.head(5))

 Now we onstructed a dataframe with all stores and all dates, it contains 91098 rows
  store_nbr       date
0         1 2013-01-02
1         1 2013-01-03
2         1 2013-01-04
3         1 2013-01-05
4         1 2013-01-06
 Now we onstructed a dataframe with all stores and all dates, it contains 91099 rows
  store_nbr       date   unit_sales   city      state type cluster
0         1 2013-01-02  7417.147949  Quito  Pichincha    D      13
1         1 2013-01-03  5873.244141  Quito  Pichincha    D      13
2         1 2013-01-04  5919.878906  Quito  Pichincha    D      13
3         1 2013-01-05  6318.785156  Quito  Pichincha    D      13
4         1 2013-01-06  2199.086914  Quito  Pichincha    D      13


In [27]:
# Merge the two dataframes and keep only the records that are in the first dataframe but not in the second dataframe
Difference_df_salesandstoresdata_alldates_df_storesanddates = df_salesandstoresdata_alldates.merge(df_storesanddates, on = ['store_nbr','date'], how='outer', indicator=True).loc[lambda x : x['_merge']=='left_only']
Difference_df_salesandstoresdata_alldates_df_storesanddates

,store_nbr,date,unit_sales,city,state,type,cluster,_merge
28679,25,2013-01-01,2511.618896,Salinas,Santa Elena,D,1,left_only


In [28]:
# Now, let's see how this works out for store number 30 (just a random one that is missing some dates according to our earlier analysis)
df_salesandstoresdata_alldates30 = df_salesandstoresdata_alldates[df_salesandstoresdata_alldates['store_nbr']=='30']
df_salesandstoresdata_alldates30 = df_salesandstoresdata_alldates30[df_salesandstoresdata_alldates30['unit_sales'].isnull()]
df_salesandstoresdata_alldates30.head(5)

,store_nbr,date,unit_sales,city,state,type,cluster
38989,30,2013-07-08,NaN,NaN,NaN,NaN,NaN
38990,30,2013-07-09,NaN,NaN,NaN,NaN,NaN
38991,30,2013-07-10,NaN,NaN,NaN,NaN,NaN
38992,30,2013-07-11,NaN,NaN,NaN,NaN,NaN
38993,30,2013-07-12,NaN,NaN,NaN,NaN,NaN


In [29]:
# Only the stores that have are value count less than 1679 in se_storedatecount
se_storedatecountmissing = se_storedatecount[se_storedatecount < 1679]

# Now, let's have df_salesandstoresdata_alldates but only for the stores where we are missing some of the data (well, atleast we miss sales on those date, we don't have them in the original data)
df_salesandstoresdata_alldatesnull = df_salesandstoresdata_alldates[df_salesandstoresdata_alldates['store_nbr'].isin(se_storedatecountmissing.index)]

# From the stores with missing data, we only want the records where the unit_sales is missing
df_salesandstoresdata_alldatesnull = df_salesandstoresdata_alldatesnull[df_salesandstoresdata_alldatesnull['unit_sales'].isnull()]

df_salesandstoresdata_alldatesnull = df_salesandstoresdata_alldatesnull[['date', 'store_nbr','unit_sales']]

# Add a unit_sales of 1 to the dataframe to make it easier to plot, it's just a dummy value
df_salesandstoresdata_alldatesnull['unit_sales'] = 1

# Merge the dataframe with the date_range dataframe to have all dates in the dataframe
df_salesandstoresdata_alldatesnull = df_storesanddates.merge(df_salesandstoresdata_alldatesnull, on=['store_nbr','date'] ,how='left')

# Now we have a dataframe with all stores and all dates, but only for the stores that are missing some data
df_salesandstoresdata_alldatesnull = df_salesandstoresdata_alldatesnull[df_salesandstoresdata_alldatesnull['store_nbr'].isin(se_storedatecountmissing.index)]

print(f"Stores {df_salesandstoresdata_alldatesnull['store_nbr'].unique()} are in the dataset with stores with <1679 datapoints and all dates, having imputed a value of 1 for all dates missing in the range")

Stores ['1' '12' '14' '17' '18' '20' '21' '22' '24' '25' '29' '30' '36' '42' '43'
 '52' '53'] are in the dataset with stores with <1679 datapoints and all dates, having imputed a value of 1 for all dates missing in the range


In [30]:
df_salesandstoresdata_alldatesnull36 = df_salesandstoresdata_alldatesnull[df_salesandstoresdata_alldatesnull['store_nbr'] == '36']
df_salesandstoresdata_alldatesnull36

,store_nbr,date,unit_sales
59045,36,2013-01-02,1.0
59046,36,2013-01-03,1.0
59047,36,2013-01-04,1.0
59048,36,2013-01-05,1.0
59049,36,2013-01-06,1.0
...,...,...,...
60727,36,2017-08-11,NaN
60728,36,2017-08-12,NaN
60729,36,2017-08-13,NaN
60730,36,2017-08-14,NaN


In [31]:
# Stores that are new we mark as 1, stores that are old we mark as 0
se_storedatecountmissingsome = se_storedatecount[se_storedatecount < 1670]

df_salesandstoresdata_alldatesnull1 = df_salesandstoresdata_alldatesnull.copy()

# Identify stores that are new based on having a dummy value on 2013-01-02
new_store_nbrs = df_salesandstoresdata_alldatesnull[
    (df_salesandstoresdata_alldatesnull['date'] == '2013-01-02') & 
    (df_salesandstoresdata_alldatesnull['unit_sales'] == 1)
]['store_nbr'].unique()

# Make a new column missingdatacategory where stores that have a dummy unit for 2013-01-02 are marked as a new store, the rest for now is seen as an old store. This is still the whole dataset 
# We get the right storenumbers based on the isin part of the expression
df_salesandstoresdata_alldatesnull1['missingdatacategory'] = np.where(df_salesandstoresdata_alldatesnull1['store_nbr'].isin(new_store_nbrs),
                                                                     'new_store', 
                                                                     'old_store'
                                                                     )

# Step 2 - For all stores that have < 1670 days of data, name the stores that are not new and old store missing > days of data
df_salesandstoresdata_alldatesnull2 = df_salesandstoresdata_alldatesnull1[df_salesandstoresdata_alldatesnull1['store_nbr'].isin(se_storedatecountmissingsome.index)]

df_salesandstoresdata_alldatesnull2['missingdatacategory'] = np.where((df_salesandstoresdata_alldatesnull2['missingdatacategory'] == 'new_store'),
                                                                    'new_store',
                                                                    'old_store missing >9 days'
                                                                    )

# Step 3 - For all stores that are missing <9 days of data we just label them "missing < 9 days"
df_salesandstoresdata_alldatesnull3 = df_salesandstoresdata_alldatesnull[~df_salesandstoresdata_alldatesnull1['store_nbr'].isin(se_storedatecountmissingsome.index)]
df_salesandstoresdata_alldatesnull3['missingdatacategory'] =        'missing <9 days'

# Put the dataframes of step 2 and 3 together to get all rows back together as in the original dataframes
df_salesandstoresdata_alldatesnullfinal = pd.concat([df_salesandstoresdata_alldatesnull2, df_salesandstoresdata_alldatesnull3])

print(df_salesandstoresdata_alldatesnull.shape)
print(df_salesandstoresdata_alldatesnull1.shape)
print(df_salesandstoresdata_alldatesnull2.shape)
print(df_salesandstoresdata_alldatesnull3.shape)

# Make a dataframe that groups the data by store and missingdatacategory
df_salesandstoresdata_alldatesnullfinal = df_salesandstoresdata_alldatesnullfinal.groupby(['store_nbr','missingdatacategory']).agg({'unit_sales':'count'}).reset_index()

# Add a dummy value to the dataframe to make it easier to plot or to join with other dataframes
df_salesandstoresdata_alldatesnullfinal['missingdata'] = '1'
df_salesandstoresdata_alldatesnullfinal

(28679, 3)
(28679, 4)
(23618, 4)
(5061, 4)


/var/folders/4z/hxnkb6_x7jn5rtztbhslhxdm0000gn/T/ipykernel_7902/2723828220.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_salesandstoresdata_alldatesnull2['missingdatacategory'] = np.where((df_salesandstoresdata_alldatesnull2['missingdatacategory'] == 'new_store'),
/var/folders/4z/hxnkb6_x7jn5rtztbhslhxdm0000gn/T/ipykernel_7902/2723828220.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_salesandstoresdata_alldatesnull3['missingdatacategory'] =        'missing <9 days'


,store_nbr,missingdatacategory,unit_sales,missingdata
0,1,missing <9 days,9,1
1,12,old_store missing >9 days,68,1
2,14,old_store missing >9 days,46,1
3,17,missing <9 days,10,1
4,18,old_store missing >9 days,118,1
5,20,new_store,776,1
6,21,new_store,937,1
7,22,new_store,1014,1
8,24,old_store missing >9 days,109,1
9,25,old_store missing >9 days,70,1


#4.0 Determine the imapct of stores

In [32]:
# Calculate the total unit sales per store from the salesandstoresdata dataframe (this original dataframe grouped the data by store and date with the original dates)
df_salesandstoresdatatotal = df_salesandstoresdata.groupby(['store_nbr']).agg({'unit_sales':'sum'}).reset_index()

# Take the df_salesandstoresdata_alldatesnullfinal dataframe and merge it with the df_salesandstoresdatatotal dataframe to get the total unit sales per store and the marking if dates are missing per store including the categories why something is missing.
df_salesandstoresdatatotal = df_salesandstoresdatatotal.merge(df_salesandstoresdata_alldatesnullfinal, on='store_nbr', how='left')

# Drop the unit_sales_y column and rename the unit_sales_x column to unit_sales (just cleaning things from the last merge)
df_salesandstoresdatatotal = df_salesandstoresdatatotal.drop(columns=['unit_sales_y'])
df_salesandstoresdatatotal = df_salesandstoresdatatotal.rename(columns={'unit_sales_x':'unit_sales'})

# If a store isn't missing data, give the missingdata column a value of 0, do the same for the missingdatacategory column
df_salesandstoresdatatotal['missingdata'] = df_salesandstoresdatatotal['missingdata'].fillna('0')
df_salesandstoresdatatotal['missingdatacategory'] = df_salesandstoresdatatotal['missingdatacategory'].fillna('0')

# Sort the dataframe by unit_sales (we want to have the highest sales first)
df_salesandstoresdatatotal = df_salesandstoresdatatotal.sort_values(by='unit_sales', ascending=False)

df_salesandstoresdatatotal

,store_nbr,unit_sales,missingdatacategory,missingdata
38,44,62087544.00,0,0
39,45,54498012.00,0,0
41,47,50948308.00,0,0
22,3,50481900.00,0,0
43,49,43420088.00,0,0
40,46,41896052.00,0,0
42,48,35933132.00,0,0
46,51,32911484.00,0,0
52,8,30491336.00,0,0
45,50,28653018.00,0,0


In [33]:
# Group the data by missingdata and missingdatacategory and calculate the percentage of the total unit sales per store
df_salesandstoresdatatotalgroupedby = df_salesandstoresdatatotal.groupby(['missingdata','missingdatacategory']).agg({'unit_sales':'sum'}).reset_index()
df_salesandstoresdatatotalgroupedby['Percentage'] = df_salesandstoresdatatotalgroupedby['unit_sales']/df_salesandstoresdatatotalgroupedby['unit_sales'].sum()*100

df_salesandstoresdatatotalgroupedby

,missingdata,missingdatacategory,unit_sales,Percentage
0,0,0,873018688.0,81.316162
1,1,missing <9 days,48567668.0,4.523770
2,1,new_store,74159080.0,6.907449
3,1,old_store missing >9 days,77864856.0,7.252618


In [34]:
# Take the df_salesandstoresdata dataframe again and filter it for the year 2017 and the month july

# Filter rows for July 2017
df_salesandstoresdata_july_2017 = df_salesandstoresdata[(df_salesandstoresdata['date'].dt.year == 2017) & (df_salesandstoresdata['date'].dt.month == 7)]

# Print the filtered DataFrame
print(df_salesandstoresdata_july_2017)

      store_nbr       date    unit_sales   city      state type cluster
1632          1 2017-07-01  11801.933594  Quito  Pichincha    D      13
1633          1 2017-07-02   5308.296875  Quito  Pichincha    D      13
1634          1 2017-07-03  12201.218750  Quito  Pichincha    D      13
1635          1 2017-07-04  10951.704102  Quito  Pichincha    D      13
1636          1 2017-07-05  14023.387695  Quito  Pichincha    D      13
...         ...        ...           ...    ...        ...  ...     ...
83586         9 2017-07-27  13218.336914  Quito  Pichincha    B       6
83587         9 2017-07-28  14229.087891  Quito  Pichincha    B       6
83588         9 2017-07-29  20919.781250  Quito  Pichincha    B       6
83589         9 2017-07-30  22259.890625  Quito  Pichincha    B       6
83590         9 2017-07-31  20525.878906  Quito  Pichincha    B       6

[1674 rows x 7 columns]


In [35]:
# Group the data by store and calculate the total unit sales per store (we repeat the same steps as we did for the whole time period)
df_salesandstoresdata_july_2017_total = df_salesandstoresdata_july_2017.groupby(['store_nbr']).agg({'unit_sales':'sum'}).reset_index()

# Take the df_salesandstoresdata_alldatesnullfinal dataframe and merge it with the df_salesandstoresdatatotal dataframe to get the total unit sales per store and the marking if dates are missing per store including the categories why something is missing.
df_salesandstoresdata_july_2017_total = df_salesandstoresdata_july_2017_total.merge(df_salesandstoresdata_alldatesnullfinal, on='store_nbr', how='left')

# Drop the unit_sales_y column and rename the unit_sales_x column to unit_sales (just cleaning things from the last merge)
df_salesandstoresdata_july_2017_total = df_salesandstoresdata_july_2017_total.drop(columns=['unit_sales_y'])
df_salesandstoresdata_july_2017_total = df_salesandstoresdata_july_2017_total.rename(columns={'unit_sales_x':'unit_sales'})

# If a store isn't missing data, give the missingdata column a value of 0, do the same for the missingdatacategory column
df_salesandstoresdata_july_2017_total['missingdata'] = df_salesandstoresdata_july_2017_total['missingdata'].fillna('0')
df_salesandstoresdata_july_2017_total['missingdatacategory'] = df_salesandstoresdata_july_2017_total['missingdatacategory'].fillna('0')

# Sort the dataframe by unit_sales (we want to have the highest sales first)
df_salesandstoresdata_july_2017_total = df_salesandstoresdata_july_2017_total.sort_values(by='unit_sales', ascending=False)

df_salesandstoresdata_july_2017_total

,store_nbr,unit_sales,missingdatacategory,missingdata
38,44,1.453714e+06,0,0
39,45,1.329123e+06,0,0
41,47,1.223399e+06,0,0
22,3,1.159554e+06,0,0
43,49,1.093551e+06,0,0
40,46,9.915211e+05,0,0
42,48,8.321640e+05,0,0
46,51,7.580400e+05,0,0
47,52,7.196889e+05,new_store,1
45,50,6.979399e+05,0,0


In [36]:
# Group the data by missingdata and missingdatacategory and calculate the percentage of the total unit sales per store
df_salesandstoresdata_july_2017_totalgroupedby = df_salesandstoresdata_july_2017_total.groupby(['missingdata','missingdatacategory']).agg({'unit_sales':'sum'}).reset_index()
df_salesandstoresdata_july_2017_totalgroupedby['Percentage'] = df_salesandstoresdata_july_2017_totalgroupedby['unit_sales']/df_salesandstoresdata_july_2017_totalgroupedby['unit_sales'].sum()*100

df_salesandstoresdata_july_2017_totalgroupedby

,missingdata,missingdatacategory,unit_sales,Percentage
0,0,0,20449800.00,75.707825
1,1,missing <9 days,1273863.75,4.716009
2,1,new_store,3443426.25,12.748012
3,1,old_store missing >9 days,1844385.00,6.828153


In [37]:
df_salesandstoresdata_july_2017_totalcopy = df_salesandstoresdata_july_2017_total.copy()

# Let's investigate the share per store type for july 2017
df_salesandstoresdata_july_2017_total_type = df_salesandstoresdata_july_2017_totalcopy.merge(df_stores, on='store_nbr', how='left')
df_salesandstoresdata_july_2017_total_type = df_salesandstoresdata_july_2017_total_type.groupby(['type']).agg({'unit_sales':'sum'}).reset_index()
df_salesandstoresdata_july_2017_total_type = df_salesandstoresdata_july_2017_total_type.sort_values(by='unit_sales', ascending=False)
df_salesandstoresdata_july_2017_total_type['Percentage'] = df_salesandstoresdata_july_2017_total_type['unit_sales']/df_salesandstoresdata_july_2017_total_type['unit_sales'].sum()*100
df_salesandstoresdata_july_2017_total_type['CumulativePercentage'] = df_salesandstoresdata_july_2017_total_type['Percentage'].cumsum()

df_salesandstoresdata_july_2017_total_type

,type,unit_sales,Percentage,CumulativePercentage
0,A,9099141.000,33.686207,33.686207
3,D,8375932.000,31.008793,64.695000
2,C,4001083.500,14.812533,79.507530
1,B,3800042.250,14.068252,93.575783
4,E,1735275.375,6.424215,100.000000


In [38]:
# Add store information (like type, cluster, state and city) to the dataframe with the total unit sales per store for July 2017
df_salesandstoresdata_july_2017_total2 = df_salesandstoresdata_july_2017_totalcopy.merge(df_stores, on='store_nbr', how='left')

# Add the total unit sales for all stores to each row of the dataframe (this makes it easier to calculate the percentage of the total unit sales per store)
df_salesandstoresdata_july_2017_total2['Total unit sales'] = df_salesandstoresdata_july_2017_total2['unit_sales'].sum()

# Calculate the percentage of the total unit sales per store
df_salesandstoresdata_july_2017_total2['Percentage'] = df_salesandstoresdata_july_2017_total2['unit_sales']/df_salesandstoresdata_july_2017_total2['Total unit sales']*100

df_salesandstoresdata_july_2017_total2

,store_nbr,unit_sales,missingdatacategory,missingdata,city,state,type,cluster,Total unit sales,Percentage
0,44,1.453714e+06,0,0,Quito,Pichincha,A,5,27011474.0,5.381841
1,45,1.329123e+06,0,0,Quito,Pichincha,A,11,27011474.0,4.920586
2,47,1.223399e+06,0,0,Quito,Pichincha,A,14,27011474.0,4.529182
3,3,1.159554e+06,0,0,Quito,Pichincha,D,8,27011474.0,4.292822
4,49,1.093551e+06,0,0,Quito,Pichincha,A,11,27011474.0,4.048469
5,46,9.915211e+05,0,0,Quito,Pichincha,A,14,27011474.0,3.670740
6,48,8.321640e+05,0,0,Quito,Pichincha,A,14,27011474.0,3.080780
7,51,7.580400e+05,0,0,Guayaquil,Guayas,A,17,27011474.0,2.806363
8,52,7.196889e+05,new_store,1,Manta,Manabi,A,11,27011474.0,2.664382
9,50,6.979399e+05,0,0,Ambato,Tungurahua,A,14,27011474.0,2.583864


In [39]:
df_salesandstoresdata_july_2017_total3 = df_salesandstoresdata_july_2017_total2.copy()

# STEP 1 - df_salesandstoresdata_july_2017_total2 only with missingdatacategory 0 or missing <9 days
df_salesandstoresdata_july_2017_total4 = df_salesandstoresdata_july_2017_total3[(df_salesandstoresdata_july_2017_total3['missingdatacategory'] == '0') | (df_salesandstoresdata_july_2017_total3['missingdatacategory'] == 'missing <9 days')]

# STEP 2 - Drop the rows where the cluster is 10 (this is a missing value)
df_salesandstoresdata_july_2017_total4 = df_salesandstoresdata_july_2017_total4[df_salesandstoresdata_july_2017_total4['cluster'] != '10']



In [40]:

df_salesandstoresdata_july_2017

,store_nbr,date,unit_sales,city,state,type,cluster
1632,1,2017-07-01,11801.933594,Quito,Pichincha,D,13
1633,1,2017-07-02,5308.296875,Quito,Pichincha,D,13
1634,1,2017-07-03,12201.218750,Quito,Pichincha,D,13
1635,1,2017-07-04,10951.704102,Quito,Pichincha,D,13
1636,1,2017-07-05,14023.387695,Quito,Pichincha,D,13
...,...,...,...,...,...,...,...
83586,9,2017-07-27,13218.336914,Quito,Pichincha,B,6
83587,9,2017-07-28,14229.087891,Quito,Pichincha,B,6
83588,9,2017-07-29,20919.781250,Quito,Pichincha,B,6
83589,9,2017-07-30,22259.890625,Quito,Pichincha,B,6


In [41]:
df_stores_f = df_salesandstoresdata_july_2017_total4[['store_nbr']]

In [42]:
filtered_df = df_salesandstoresdata_july_2017[df_salesandstoresdata_july_2017['store_nbr'].isin(df_stores_f['store_nbr'])]
print(filtered_df)


      store_nbr       date    unit_sales   city      state type cluster
1632          1 2017-07-01  11801.933594  Quito  Pichincha    D      13
1633          1 2017-07-02   5308.296875  Quito  Pichincha    D      13
1634          1 2017-07-03  12201.218750  Quito  Pichincha    D      13
1635          1 2017-07-04  10951.704102  Quito  Pichincha    D      13
1636          1 2017-07-05  14023.387695  Quito  Pichincha    D      13
...         ...        ...           ...    ...        ...  ...     ...
83586         9 2017-07-27  13218.336914  Quito  Pichincha    B       6
83587         9 2017-07-28  14229.087891  Quito  Pichincha    B       6
83588         9 2017-07-29  20919.781250  Quito  Pichincha    B       6
83589         9 2017-07-30  22259.890625  Quito  Pichincha    B       6
83590         9 2017-07-31  20525.878906  Quito  Pichincha    B       6

[1116 rows x 7 columns]


In [43]:
df_salesdata.head()

,id,store_nbr,item_nbr,unit_sales,date
0,0,25,103665,7.0,2013-01-01
1,1,25,105574,1.0,2013-01-01
2,2,25,105575,2.0,2013-01-01
3,3,25,108079,1.0,2013-01-01
4,4,25,108701,1.0,2013-01-01


In [44]:
# Filter the df_salesdata DataFrame for rows where 'store_nbr' is in df_stores_f
df_filtered_salesdata = df_salesdata[df_salesdata['store_nbr'].isin(df_stores_f['store_nbr'])]

# Display the filtered DataFrame
print(df_filtered_salesdata)

                  id store_nbr  item_nbr  unit_sales       date
578              578         1    103665         2.0 2013-01-02
579              579         1    105574         8.0 2013-01-02
580              580         1    105575        15.0 2013-01-02
581              581         1    105577         2.0 2013-01-02
582              582         1    105737         2.0 2013-01-02
...              ...       ...       ...         ...        ...
125497035  125497035        54   2089339         4.0 2017-08-15
125497036  125497036        54   2106464         1.0 2017-08-15
125497037  125497037        54   2110456       192.0 2017-08-15
125497038  125497038        54   2113914       198.0 2017-08-15
125497039  125497039        54   2116416         2.0 2017-08-15

[93346220 rows x 5 columns]


In [45]:
# Filter the df_salesdata DataFrame for rows where 'store_nbr' is in df_stores_f and date is in July 2017
filtered_df_salesdata_july2017 = df_filtered_salesdata[
   
    (df_filtered_salesdata['date'] >= '2017-07-01') & 
    (df_filtered_salesdata['date'] <= '2017-07-31')
]

# Display the filtered DataFrame
filtered_df_salesdata_july2017

,id,store_nbr,item_nbr,unit_sales,date
120642911,120642911,1,99197,2.0,2017-07-01
120642912,120642912,1,103520,2.0,2017-07-01
120642913,120642913,1,103665,11.0,2017-07-01
120642914,120642914,1,105574,2.0,2017-07-01
120642915,120642915,1,105575,3.0,2017-07-01
...,...,...,...,...,...
123926067,123926067,54,2105347,2.0,2017-07-31
123926068,123926068,54,2106464,1.0,2017-07-31
123926069,123926069,54,2110456,577.0,2017-07-31
123926070,123926070,54,2113914,13.0,2017-07-31


In [ ]:
df_items.head(5)

In [46]:
# Merge the filtered sales data with the items data
merged_df = pd.merge(filtered_df_salesdata_july2017, df_items, on="item_nbr", how="inner")

# Display the merged DataFrame
print(merged_df)

                id store_nbr  item_nbr  unit_sales       date        family  \
0        120642911         1     99197         2.0 2017-07-01     GROCERY I   
1        120642912         1    103520         2.0 2017-07-01     GROCERY I   
2        120642913         1    103665        11.0 2017-07-01  BREAD/BAKERY   
3        120642914         1    105574         2.0 2017-07-01     GROCERY I   
4        120642915         1    105575         3.0 2017-07-01     GROCERY I   
...            ...       ...       ...         ...        ...           ...   
2265458  123926067        54   2105347         2.0 2017-07-31        BEAUTY   
2265459  123926068        54   2106464         1.0 2017-07-31     BEVERAGES   
2265460  123926069        54   2110456       577.0 2017-07-31     BEVERAGES   
2265461  123926070        54   2113914        13.0 2017-07-31      CLEANING   
2265462  123926071        54   2116416         4.0 2017-07-31     GROCERY I   

         class  perishable  
0         1067        

In [47]:
# Calculate total sales for each family
family_sales = merged_df.groupby('family')['unit_sales'].sum()

# Calculate total sales across all families
total_sales = family_sales.sum()

# Calculate the percentage of total sales for each family
family_sales_percentage = (family_sales / total_sales) * 100

# Convert the result to a DataFrame for better readability
family_sales_percentage_df = family_sales_percentage.reset_index()
family_sales_percentage_df.columns = ['family', 'percentage_of_total_sales']

# Sort the DataFrame by percentage in descending order
family_sales_percentage_df = family_sales_percentage_df.sort_values(by='percentage_of_total_sales', ascending=False)

# Display the result
print(family_sales_percentage_df)

                        family  percentage_of_total_sales
12                   GROCERY I                  28.792027
3                    BEVERAGES                  22.918161
30                     PRODUCE                  15.333992
7                     CLEANING                   8.287594
8                        DAIRY                   5.572891
5                 BREAD/BAKERY                   3.303065
28                     POULTRY                   2.690466
24                       MEATS                   2.487304
18                   HOME CARE                   2.097467
25               PERSONAL CARE                   1.996807
9                         DELI                   1.931723
10                        EGGS                   1.230737
11                FROZEN FOODS                   0.838319
22            LIQUOR,WINE,BEER                   0.577699
29              PREPARED FOODS                   0.549819
13                  GROCERY II                   0.232890
16         HOM

In [49]:
# Calculate total sales for each family
total_sales_per_family = merged_df.groupby('family')['unit_sales'].sum()

# Calculate overall total sales
overall_total_sales = total_sales_per_family.sum()

# Calculate percentage of total sales for each family
sales_percentage_per_family = (total_sales_per_family / overall_total_sales) * 100

# Group by 'family' and count distinct 'store_nbr'
distinct_stores_per_family = merged_df.groupby('family')['store_nbr'].nunique()

# Convert results to DataFrames for merging
distinct_stores_df = distinct_stores_per_family.reset_index()
distinct_stores_df.columns = ['family', 'distinct_stores_count']

sales_percentage_df = sales_percentage_per_family.reset_index()
sales_percentage_df.columns = ['family', 'sales_percentage']

# Merge the two DataFrames
result_df = pd.merge(distinct_stores_df, sales_percentage_df, on='family')

# Sort by the percentage of sales in descending order
result_df = result_df.sort_values(by='sales_percentage', ascending=False)

# Display the result
print(result_df)

                        family  distinct_stores_count  sales_percentage
12                   GROCERY I                     36         28.792027
3                    BEVERAGES                     36         22.918161
30                     PRODUCE                     36         15.333992
7                     CLEANING                     36          8.287594
8                        DAIRY                     36          5.572891
5                 BREAD/BAKERY                     36          3.303065
28                     POULTRY                     36          2.690466
24                       MEATS                     36          2.487304
18                   HOME CARE                     36          2.097467
25               PERSONAL CARE                     36          1.996807
9                         DELI                     36          1.931723
10                        EGGS                     36          1.230737
11                FROZEN FOODS                     36          0

 So, as we can see all WE can removie baby care and books as they are not seen in all stores. We lose 0.02. Now we want to look on items level 

In [50]:


# Calculate total sales for each item
total_sales_per_item = merged_df.groupby('item_nbr')['unit_sales'].sum()

# Calculate overall total sales
overall_total_sales = total_sales_per_item.sum()

# Calculate percentage of total sales for each item
sales_percentage_per_item = (total_sales_per_item / overall_total_sales) * 100

# Count distinct stores for each item
distinct_stores_per_item = merged_df.groupby('item_nbr')['store_nbr'].nunique()

# Convert results to DataFrames for merging
distinct_stores_df = distinct_stores_per_item.reset_index()
distinct_stores_df.columns = ['item_nbr', 'distinct_stores_count']

sales_percentage_df = sales_percentage_per_item.reset_index()
sales_percentage_df.columns = ['item_nbr', 'sales_percentage']

# Merge the two DataFrames
result_df = pd.merge(distinct_stores_df, sales_percentage_df, on='item_nbr')

# Sort by sales percentage in descending order
result_df = result_df.sort_values(by='sales_percentage', ascending=False)

# Display the result
print(result_df)

result_df

      item_nbr  distinct_stores_count  sales_percentage
2706   1503844                     23          0.862514
3660   2042941                     36          0.750663
3661   2042947                     36          0.635435
2636   1473474                     35          0.601705
3415   2006812                     21          0.593996
...        ...                    ...               ...
2141   1370565                      1          0.000005
1845   1239794                      1          0.000005
739     603255                      1          0.000005
1918   1243803                      1          0.000005
247     269029                      1         -0.000298

[3839 rows x 3 columns]


,item_nbr,distinct_stores_count,sales_percentage
2706,1503844,23,0.862514
3660,2042941,36,0.750663
3661,2042947,36,0.635435
2636,1473474,35,0.601705
3415,2006812,21,0.593996
...,...,...,...
2141,1370565,1,0.000005
1845,1239794,1,0.000005
739,603255,1,0.000005
1918,1243803,1,0.000005


In [51]:

# Ensure 'date' column is datetime type
merged_df['date'] = pd.to_datetime(merged_df['date'])

# Define the range of dates (July 2017 in this case)
date_range = pd.date_range(start='2017-07-01', end='2017-07-31')

# Get unique store and item numbers
stores = merged_df['store_nbr'].unique()
items = merged_df['item_nbr'].unique()

# Create a DataFrame with all combinations of store, item, and date
all_combinations = pd.MultiIndex.from_product([stores, items, date_range], names=['store_nbr', 'item_nbr', 'date']).to_frame(index=False)

# Merge with the original DataFrame
df_full = pd.merge(all_combinations, merged_df, on=['store_nbr', 'item_nbr', 'date'], how='left')

# Fill NaN values in 'unit_sales' with 0
df_full['unit_sales'] = df_full['unit_sales'].fillna(0)

# Optionally, reset the index if needed
df_full.reset_index(drop=True, inplace=True)

# Display the balanced DataFrame
print(df_full)


        store_nbr  item_nbr       date           id  unit_sales     family  \
0               1     99197 2017-07-01  120642911.0         2.0  GROCERY I   
1               1     99197 2017-07-02          NaN         0.0        NaN   
2               1     99197 2017-07-03          NaN         0.0        NaN   
3               1     99197 2017-07-04          NaN         0.0        NaN   
4               1     99197 2017-07-05          NaN         0.0        NaN   
...           ...       ...        ...          ...         ...        ...   
4284319        54   1066900 2017-07-27          NaN         0.0        NaN   
4284320        54   1066900 2017-07-28          NaN         0.0        NaN   
4284321        54   1066900 2017-07-29          NaN         0.0        NaN   
4284322        54   1066900 2017-07-30          NaN         0.0        NaN   
4284323        54   1066900 2017-07-31          NaN         0.0        NaN   

          class  perishable  
0        1067.0         0.0  
1  

In [52]:
# Ensure 'date' column is datetime type
df_full['date'] = pd.to_datetime(df_full['date'])

# Step 1: Filter out rows where 'unit_sales' is zero or NaN
df_positive_sales = df_full[df_full['unit_sales'] > 0]

# Step 2: Group by store and item, then count unique dates with sales > 0
daily_sales_counts = df_positive_sales.groupby(['store_nbr', 'item_nbr'])['date'].nunique().reset_index()
daily_sales_counts.rename(columns={'date': 'days_with_sales'}, inplace=True)

# Step 3: Calculate the average number of days with sales > 0 per item
average_daily_sales = daily_sales_counts.groupby(['item_nbr'])['days_with_sales'].mean().reset_index()

# Step 4: Calculate total sales for each item across all stores and dates
total_sales_per_item = df_full.groupby('item_nbr')['unit_sales'].sum().reset_index()
total_sales_per_item.rename(columns={'unit_sales': 'total_sales'}, inplace=True)

# Step 5: Calculate total sales for all items
grand_total_sales = total_sales_per_item['total_sales'].sum()

# Step 6: Calculate the percentage of sales for each item
total_sales_per_item['sales_percentage'] = (total_sales_per_item['total_sales'] / grand_total_sales) * 100

# Step 7: Merge the percentage information with the average daily sales
average_daily_sales = average_daily_sales.merge(total_sales_per_item[['item_nbr', 'sales_percentage']], on='item_nbr')

# Step 8: Print or inspect the results
print(average_daily_sales)

      item_nbr  days_with_sales  sales_percentage
0        96995        11.000000          0.001540
1        99197        15.684211          0.005618
2       103501        28.400000          0.008225
3       103520        21.944444          0.012542
4       103665        26.230769          0.013401
...        ...              ...               ...
3833   2116416        13.636364          0.015298
3834   2118662         7.000000          0.000114
3835   2122188         3.000000          0.000969
3836   2124052        13.000000          0.010431
3837   2127114         6.411765          0.003144

[3838 rows x 3 columns]


In [ ]:
import pandas as pd



# Ensure 'date' column is datetime type
df_full['date'] = pd.to_datetime(df_full['date'])

# Filter out rows where 'unit_sales' is zero or NaN
df_positive_sales = df_full[df_full['unit_sales'] > 0]

# Group by store and item, then count unique dates with sales > 0
daily_sales_counts = df_positive_sales.groupby(['store_nbr', 'item_nbr'])['date'].nunique().reset_index()
daily_sales_counts.rename(columns={'date': 'days_with_sales'}, inplace=True)

# Calculate the average number of days with sales > 0 per item
average_daily_sales = daily_sales_counts.groupby(['item_nbr'])['days_with_sales'].mean().reset_index()

# Calculate total sales for each item across all stores and dates
total_sales_per_item = df_full.groupby('item_nbr')['unit_sales'].sum().reset_index()
total_sales_per_item.rename(columns={'unit_sales': 'total_sales'}, inplace=True)

# Calculate total sales for all items
grand_total_sales = total_sales_per_item['total_sales'].sum()

# Calculate the percentage of sales for each item
total_sales_per_item['sales_percentage'] = (total_sales_per_item['total_sales'] / grand_total_sales) * 100

# Merge the percentage information with the average daily sales
average_daily_sales = average_daily_sales.merge(total_sales_per_item[['item_nbr', 'sales_percentage']], on='item_nbr')

# Print basic statistics for average_daily_sales
print("Basic Statistics for `average_daily_sales` DataFrame:")
print(average_daily_sales.describe())

# View the first few rows
print("\nFirst few rows of `average_daily_sales` DataFrame:")
print(average_daily_sales.head())

# View the last few rows
print("\nLast few rows of `average_daily_sales` DataFrame:")
print(average_daily_sales.tail()) 

# DataFrame info
print("\nInfo on `average_daily_sales` DataFrame:")
print(average_daily_sales.info())


In [ ]:



# Ensure 'date' column is datetime type
df_full['date'] = pd.to_datetime(df_full['date'])

# Filter out rows where 'unit_sales' is zero or NaN
df_positive_sales = df_full[df_full['unit_sales'] > 0]

# Group by store and item, then count unique dates with sales > 0
daily_sales_counts = df_positive_sales.groupby(['store_nbr', 'item_nbr'])['date'].nunique().reset_index()
daily_sales_counts.rename(columns={'date': 'days_with_sales'}, inplace=True)

# Calculate the average number of days with sales > 0 per item
average_daily_sales = daily_sales_counts.groupby(['item_nbr'])['days_with_sales'].mean().reset_index()

# Calculate total sales for each item across all stores and dates
total_sales_per_item = df_full.groupby('item_nbr')['unit_sales'].sum().reset_index()
total_sales_per_item.rename(columns={'unit_sales': 'total_sales'}, inplace=True)

# Calculate total sales for all items
grand_total_sales = total_sales_per_item['total_sales'].sum()

# Calculate the percentage of sales for each item
total_sales_per_item['sales_percentage'] = (total_sales_per_item['total_sales'] / grand_total_sales) * 100

# Merge the percentage information with the average daily sales
average_daily_sales = average_daily_sales.merge(total_sales_per_item[['item_nbr', 'sales_percentage']], on='item_nbr')

# Filter items that are sold for fewer than 10 days on average
items_sold_fewer_than_10_days = average_daily_sales[average_daily_sales['days_with_sales'] < 10]

# Calculate the total number of items
total_items = average_daily_sales.shape[0]

# Calculate the number of items sold for fewer than 10 days
items_sold_fewer_than_10_days_count = items_sold_fewer_than_10_days.shape[0]

# Compute the percentage
percentage_sold_fewer_than_10_days = (items_sold_fewer_than_10_days_count / total_items) * 100

# Print the result
print(f"Total percentage of items sold for fewer than 10 days on average: {percentage_sold_fewer_than_10_days:.2f}%")
import pandas as pd

# Sample DataFrame setup (Replace this with loading your actual DataFrame)
# df_full = pd.read_csv('your_data.csv')

# Ensure 'date' column is datetime type
df_full['date'] = pd.to_datetime(df_full['date'])

# Filter out rows where 'unit_sales' is zero or NaN
df_positive_sales = df_full[df_full['unit_sales'] > 0]

# Group by store and item, then count unique dates with sales > 0
daily_sales_counts = df_positive_sales.groupby(['store_nbr', 'item_nbr'])['date'].nunique().reset_index()
daily_sales_counts.rename(columns={'date': 'days_with_sales'}, inplace=True)

# Calculate the average number of days with sales > 0 per item
average_daily_sales = daily_sales_counts.groupby(['item_nbr'])['days_with_sales'].mean().reset_index()

# Calculate total sales for each item across all stores and dates
total_sales_per_item = df_full.groupby('item_nbr')['unit_sales'].sum().reset_index()
total_sales_per_item.rename(columns={'unit_sales': 'total_sales'}, inplace=True)

# Calculate total sales for all items
grand_total_sales = total_sales_per_item['total_sales'].sum()

# Calculate the percentage of sales for each item
total_sales_per_item['sales_percentage'] = (total_sales_per_item['total_sales'] / grand_total_sales) * 100

# Merge the percentage information with the average daily sales
average_daily_sales = average_daily_sales.merge(total_sales_per_item[['item_nbr', 'sales_percentage']], on='item_nbr')

# Filter items that are sold for fewer than 10 days on average
items_sold_fewer_than_10_days = average_daily_sales[average_daily_sales['days_with_sales'] < 10]

# Calculate the total number of items
total_items = average_daily_sales.shape[0]

# Calculate the number of items sold for fewer than 10 days
items_sold_fewer_than_10_days_count = items_sold_fewer_than_10_days.shape[0]

# Compute the percentage
percentage_sold_fewer_than_10_days = (items_sold_fewer_than_10_days_count / total_items) * 100

# Print the result
print(f"Total percentage of items sold for fewer than 10 days on average: {percentage_sold_fewer_than_10_days:.2f}%")




In [ ]:
import pandas as pd

# Sample DataFrame setup (Replace this with loading your actual DataFrame)
# df_full = pd.read_csv('your_data.csv')

# Ensure 'date' column is datetime type
df_full['date'] = pd.to_datetime(df_full['date'])

# Filter out rows where 'unit_sales' is zero or NaN
df_positive_sales = df_full[df_full['unit_sales'] > 0]

# Group by store and item, then count unique dates with sales > 0
daily_sales_counts = df_positive_sales.groupby(['store_nbr', 'item_nbr'])['date'].nunique().reset_index()
daily_sales_counts.rename(columns={'date': 'days_with_sales'}, inplace=True)

# Calculate the average number of days with sales > 0 per item
average_daily_sales = daily_sales_counts.groupby(['item_nbr'])['days_with_sales'].mean().reset_index()

# Calculate total sales for each item across all stores and dates
total_sales_per_item = df_full.groupby('item_nbr')['unit_sales'].sum().reset_index()
total_sales_per_item.rename(columns={'unit_sales': 'total_sales'}, inplace=True)

# Calculate total sales for all items
grand_total_sales = total_sales_per_item['total_sales'].sum()

# Calculate the percentage of sales for each item
total_sales_per_item['sales_percentage'] = (total_sales_per_item['total_sales'] / grand_total_sales) * 100

# Merge the percentage information with the average daily sales
average_daily_sales = average_daily_sales.merge(total_sales_per_item[['item_nbr', 'sales_percentage']], on='item_nbr')

# Calculate the total number of days in the period
total_days = (df_full['date'].max() - df_full['date'].min()).days + 1

# Calculate the number of weeks in the period
total_weeks = total_days / 7

# Determine the number of days with sales data for each item
item_sales_per_week = daily_sales_counts.groupby('item_nbr')['days_with_sales'].sum() / total_weeks

# Determine if the item is sold at least once a week on average
items_sold_at_least_once_a_week = item_sales_per_week[item_sales_per_week >= 1].index

# Calculate the total number of items
total_items = average_daily_sales.shape[0]

# Calculate the number of items sold at least once a week
items_sold_at_least_once_a_week_count = len(items_sold_at_least_once_a_week)

# Compute the percentage
percentage_sold_at_least_once_a_week = (items_sold_at_least_once_a_week_count / total_items) * 100

# Print the result
print(f"Total percentage of items sold at least once a week on average: {percentage_sold_at_least_once_a_week:.2f}%")
